In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
import keras

Using TensorFlow backend.


In [5]:
from keras import backend as K

In [6]:
from keras.models import Sequential

In [7]:
from keras.layers import Dense

In [8]:
from sklearn import model_selection

In [9]:
import math

In [10]:
import tensorflow as tf

In [11]:
import horovod.keras as hvd

In [12]:
df = pd.DataFrame()

In [13]:
df['distances'] = np.array([np.random.randint(1,1000) for _ in range(10**5)])

In [14]:
df['weights'] = np.array([np.random.randint(60,500) for _ in range(10**5)])

In [15]:
df[['distances','weights']].count()

distances    100000
weights      100000
dtype: int64

In [16]:
df['prices'] = np.array([((df['distances'][i] * df['weights'][i])/100) for i in range(len(df['distances']))])

In [17]:
df.head()

,distances,weights,prices
0,494,410,2025.40
1,378,140,529.20
2,768,361,2772.48
3,534,222,1185.48
4,610,293,1787.30


In [18]:
hvd.init()

0

In [19]:
config = tf.ConfigProto()

In [20]:
config.gpu_options.allow_growth = True

In [21]:
config.gpu_options.visible_device_list = str(hvd.local_rank())

In [22]:
K.set_session(tf.Session(config=config))

In [23]:
batch_size = 128

In [24]:
num_classes = 10

In [25]:
epochs = int(math.ceil(100.0) / hvd.size())

In [26]:
x, x_val, y, y_val = model_selection.train_test_split(df['prices'], df['distances'], test_size=0.2, random_state=42)

In [27]:
dlce = lambda l: [v for v in l if v >= 0]

In [28]:
x = np.array(dlce(x))
x_val = np.array(dlce(x_val))

In [29]:
y = np.array(dlce(y))
y_val = np.array(dlce(y_val))

In [30]:
opt = keras.optimizers.RMSprop(1.0 * hvd.size())

In [31]:
opt = hvd.DistributedOptimizer(opt)

In [32]:
model = Sequential()

In [33]:
model.add(Dense(1, input_dim=1))

In [34]:
model.compile(optimizer=opt, loss=keras.losses.mse,metrics=['accuracy'])

In [35]:
callbacks = [hvd.callbacks.BroadcastGlobalVariablesCallback(0)]

In [ ]:
if hvd.rank() == 0:
    callbacks.append(keras.callbacks.ModelCheckpoint('./checkpoint-{epoch}.h5'))

In [ ]:
model.fit(x, y, batch_size=batch_size, callbacks=callbacks, epochs=epochs, verbose=1, validation_data=(x_val, y_val))

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 1s 7us/step - loss: 822385.8785 - acc: 0.0012 - val_loss: 66478.1534 - val_acc: 0.0031
Epoch 2/100
80000/80000 [==============================] - 1s 7us/step - loss: 806710.5962 - acc: 8.0000e-04 - val_loss: 216395.0884 - val_acc: 0.0011
Epoch 3/100
80000/80000 [==============================] - 1s 7us/step - loss: 808642.8604 - acc: 9.1250e-04 - val_loss: 731877.2580 - val_acc: 3.5000e-04
Epoch 4/100
80000/80000 [==============================] - 1s 7us/step - loss: 796000.7692 - acc: 8.7500e-04 - val_loss: 259438.0246 - val_acc: 7.5000e-04
Epoch 5/100
80000/80000 [==============================] - 1s 7us/step - loss: 795769.7249 - acc: 7.1250e-04 - val_loss: 526123.9369 - val_acc: 6.0000e-04
Epoch 6/100
80000/80000 [==============================] - 1s 8us/step - loss: 809484.7026 - acc: 5.6250e-04 - val_loss: 351436.9450 - val_acc: 7.5000e-04
Epoch 7/100
80000/80000 [========

80000/80000 [==============================] - 1s 7us/step - loss: 798058.8412 - acc: 7.0000e-04 - val_loss: 164559.8771 - val_acc: 5.5000e-04
Epoch 54/100
80000/80000 [==============================] - 1s 8us/step - loss: 792952.7757 - acc: 5.5000e-04 - val_loss: 2120055.4472 - val_acc: 3.0000e-04
Epoch 55/100
80000/80000 [==============================] - 1s 8us/step - loss: 813691.4412 - acc: 5.5000e-04 - val_loss: 488700.2658 - val_acc: 7.0000e-04
Epoch 56/100
80000/80000 [==============================] - 1s 8us/step - loss: 794372.5686 - acc: 6.2500e-04 - val_loss: 573928.4403 - val_acc: 0.0010
Epoch 57/100
80000/80000 [==============================] - 1s 8us/step - loss: 798444.5675 - acc: 5.2500e-04 - val_loss: 906252.3244 - val_acc: 4.0000e-04
Epoch 58/100
80000/80000 [==============================] - 1s 8us/step - loss: 803538.4639 - acc: 6.2500e-04 - val_loss: 564356.7012 - val_acc: 5.0000e-04
Epoch 59/100
80000/80000 [==============================] - 1s 8us/step - loss: 

In [ ]:
score = model.evaluate(x_val, y_val, verbose=0)

In [ ]:
print('Test loss:', score[0])

In [ ]:
print('Test accuracy:', score[1])

In [ ]:
print(model.predict(np.array([np.random.randint(10,100) for _ in range(5)])))

In [ ]:
model.save_weights('travel-price.hdf5')

In [ ]:
with open('travel-price.json','w') as f:
    f.write(model.to_json())
    f.close()